In [1]:
from tqdm import tqdm
from collections import Counter
import random
from copy import copy
import numpy as np

In [2]:
FILE_NAME="edges.txt"

In [3]:
%%bash
FILE_NAME="edges.txt"
LINK="https://d3c33hcgiwev3.cloudfront.net/_d4f3531eac1d289525141e95a2fea52f_edges.txt?Expires=1616284800&Signature=WexgAzJvRL2d8C8PTcWWXm3wHdXu3yLvUtsFU7SC8Ap5iyzm63G65m4NMcsGYbVUzXO9c2GdNw3yEpav3BUOqQZqloMHfU92ltKp1m0vUIWerCxbhmlnoJpO78p8kuUv9uyzoBugC9onSFKmUxnmVp-R~NGe7dPAHMq5Gjqwv6s_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

wget -O $FILE_NAME $LINK 

--2021-03-19 17:51:02--  https://d3c33hcgiwev3.cloudfront.net/_d4f3531eac1d289525141e95a2fea52f_edges.txt?Expires=1616284800&Signature=WexgAzJvRL2d8C8PTcWWXm3wHdXu3yLvUtsFU7SC8Ap5iyzm63G65m4NMcsGYbVUzXO9c2GdNw3yEpav3BUOqQZqloMHfU92ltKp1m0vUIWerCxbhmlnoJpO78p8kuUv9uyzoBugC9onSFKmUxnmVp-R~NGe7dPAHMq5Gjqwv6s_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 52.85.114.40, 52.85.114.22, 52.85.114.74, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|52.85.114.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28291 (28K) [text/plain]
Saving to: ‘edges.txt’

     0K .......... .......... .......                         100% 3,09M=0,009s

2021-03-19 17:51:03 (3,09 MB/s) - ‘edges.txt’ saved [28291/28291]



In [3]:
! ls -lh

total 96
-rw-r--r--  1 Alexander  staff    28K Mar 29  2016 edges.txt
-rw-r--r--  1 Alexander  staff    17K Mar 22 00:47 prim_min_span_tree.ipynb


In [20]:
with open(FILE_NAME) as file:
    a = file.read().splitlines()

In [21]:
n_nodes, n_endes = tuple(map(int, a[0].split(' ')))

In [22]:
edges[:10]

[(1, 2, 6807),
 (2, 3, -8874),
 (3, 4, -1055),
 (4, 5, 4414),
 (5, 6, 1728),
 (6, 7, -2237),
 (7, 8, -7507),
 (8, 9, 7990),
 (9, 10, -5012),
 (10, 11, 7353)]

In [ ]:
from dataclasses import dataclass

@dataclass 
class Edge:
    src: int
    dst: int
    weight: int
        
    # fix for python heap when sorting tuples 
    def __le__(self,other):
        return True
    
    def __lt__(self, other):
        return True
          
@dataclass
class Graph:
    V: set
    v_to_edgelist: dict
        

V = set()
v_to_edgelist = {}
for edge in edges:
    e = Edge(src=edge[0], dst=edge[1], weight=edge[2])
    e2 = Edge(src=edge[1], dst=edge[0], weight=edge[2])
    if e.src in v_to_edgelist:
        v_to_edgelist[e.src].append(e)
    else:
        v_to_edgelist[e.src] = [e]
        
    if e2.src in v_to_edgelist:
        v_to_edgelist[e2.src].append(e2)
    else:
        v_to_edgelist[e2.src] = [e2]
    
    V.add(e.src)
    V.add(e.dst)
    
G = Graph(V=V, v_to_edgelist=v_to_edgelist)


# Prim

In [71]:
import heapq

def prim_mst(G, start_v):
    mst_cost = 0
    mst = []
    curr_v = start_v
    heap = []
    X = set([curr_v]) # explored

    while len(X) != len(G.V):
        for edge in G.v_to_edgelist[curr_v]:
            if edge.dst not in X:
                heapq.heappush(heap, (edge.weight, edge))
        
        weight, edge = heapq.heappop(heap)
        while edge.dst in X:
            weight, edge = heapq.heappop(heap)
        mst_cost += weight
        mst.append(edge)
        X.add(edge.dst)
        curr_v = edge.dst
    return mst_cost, mst

In [72]:
mst_cost, mst = prim_mst(G, 1)
mst_cost

-3612829

# Kruskal

https://github.com/sh2439/Algo_stanford/blob/master/C3%3A%20Greedy%20Algo/clustering_small.py \
https://algotree.org/algorithms/minimum_spanning_tree/kruskals/

In [74]:
class UnionFind:
    def __init__(self,n):
        self.parent = list(range(n))
        self.rank = [0] * n
        # self.n_clusters = n
    
    def find(self, x):
        if self.parent[x] != x:
            # path compression
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def count(self):
        return self.n_clusters
        
    def union(self, x,y):
        root1 = self.find(x)
        root2 = self.find(y)
        
        if root1 == root2:
            return False
        
        # self.n_clusters -= 1
        # union by rank
        if self.rank[root1] == self.rank[root2]:
            self.parent[root2] = root1
            self.rank[root1] += 1
        elif self.rank[root1] > self.rank[root2]:
            self.parent[root2] = root1
        else:
            self.parent[root1] = root2
        return True
            
    def connected(self, x, y):
        return self.find(x) == self.find(y)

In [76]:
from dataclasses import dataclass
@dataclass
class Edge:
    src: int
    dst: int
    weight: int
        
    
def kruskal_mst(num_nodes, edgelist):
    mst = []
    mst_cost = 0
    # Sort objects of an Edge class based on attribute (weight)
    edgelist.sort(key=lambda Edge : Edge.weight)

    U = UnionFind(num_nodes)
    # working on sorted list
    for edge in tqdm(edgelist):
        united = U.union(edge.src, edge.dst)
        if united:
            mst.append(edge)
            mst_cost += edge.weight
    return mst_cost, mst

In [79]:
%%time
# subtract one so vertices start from 0
mst_cost, mst = kruskal_mst(n_nodes, [Edge(x[0]-1, x[1]-1, x[2]) for x in edges])
mst_cost

100%|██████████| 2184/2184 [00:00<00:00, 275036.33it/s]

CPU times: user 11.4 ms, sys: 2.16 ms, total: 13.5 ms
Wall time: 13.4 ms


-3612829

# Trash

In [30]:
e_orig = [Edge(np.random.randint(0,100000), np.random.randint(0,100000), np.random.randint(-1000,1000)) for i in range(1_000_000)]

In [31]:
%%time
e = copy(e_orig)
g1 = Graph(len(e), e)
g1.KruskalMST()

100%|██████████| 1000000/1000000 [00:03<00:00, 283611.03it/s]

Edges of minimum spanning tree are in Graph 1 : 
Cost of minimum spanning tree : -88038421
5836773
CPU times: user 4.11 s, sys: 67.7 ms, total: 4.18 s
Wall time: 4.18 s


In [33]:
%%time
e = copy(e_orig)
g1 = Graph(len(e), e)
g1.KruskalMST()

100%|██████████| 1000000/1000000 [00:02<00:00, 381272.25it/s]

Edges of minimum spanning tree are in Graph 1 : 
Cost of minimum spanning tree : -88038421
3951267
CPU times: user 3.2 s, sys: 49.3 ms, total: 3.25 s
Wall time: 3.25 s


In [42]:
# num_nodes = 6
# e1 = Edge(0, 1, 4);
# e2 = Edge(0, 2, 1);
# e3 = Edge(0, 3, 5);
# e4 = Edge(1, 3, 2);
# e5 = Edge(1, 4, 3);
# e6 = Edge(1, 5, 3);
# e7 = Edge(2, 3, 2);
# e8 = Edge(2, 4, 8);
# e9 = Edge(3, 4, 1);
# e10 = Edge(4, 5, 3);

# g1 = Graph(num_nodes, [e1, e2, e3, e4, e5, e6, e7, e8, e9, e10])
# g1.KruskalMST()

Edges of minimum spanning tree are in Graph 1 : 
Cost of minimum spanning tree : 9


In [51]:
with open('../scc/SCC.txt') as file:
    a = file.read().splitlines()

In [55]:
b = [x.split(' ') for x in a]

In [57]:
c = [(int(x[0]), int(x[1])) for x in b]

In [65]:
import numpy as np

In [72]:
many_edges = [Edge(x[0]-1, x[1]-1, np.random.randint(-1000, 1000)) for x in c]

In [82]:
e = copy(many_edges)
g1 = Graph(len(e), e)

In [83]:
%%time
g1.KruskalMST()
counter

started


KeyboardInterrupt: 

In [75]:
len(e)

5105043